In [1]:
import os
#import cv2
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import tensorflow as tf
from keras import optimizers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input , Dense , Dropout , Flatten,\
Conv2D,MaxPooling2D , BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
#import tensorflow_addons as tfa

In [2]:
path = 'C://Users//User//Desktop//Python//Hamoye//Stage D//planet//planet//'
os.listdir(path)

['sample_submission.csv',
 'test-jpg',
 'train-jpg',
 'train_classes.csv',
 'train_v2.csv']

In [3]:
train_path = os.path.join(path,"train_classes.csv")
train_dir = os.path.join(path,'train-jpg')
submission_df = os.path.join(path,'sample_submission.csv')
test_dir = os.path.join(path,'test-jpg')

In [4]:
df_train = pd.read_csv(train_path)
submission_df = pd.read_csv(submission_df)

In [5]:
def preprocessing(data):
    
    # To Get all unique Labels (Target)
    labels = data['tags'].to_numpy()
    unique_labels = list()
    for label in labels:
        tags = label.split()
        for tag in tags:
            if tag not in unique_labels:
                unique_labels.append(tag)
                
                
    # Encoding Classes (targets)
    for unique_label in unique_labels:
        data[unique_label]=data["tags"].apply(lambda x: 1 if unique_label in x.split() else 0)
    
    # Adding .jpg to each images name to be able to iterate through them
    data['image_name'] = data['image_name'].apply(lambda x: x+".jpg")
    return data 

In [6]:
df_train = preprocessing(df_train)

In [7]:
df_train.head(3)

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0.jpg,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1.jpg,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2.jpg,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
model = Sequential()
#input layer
model.add(BatchNormalization(input_shape=(128, 128, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

#Hidden layers
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

#Output layer
model.add(Dense(17, activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 128, 128, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        1

In [10]:
train_datagenerator = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

train_datagen = train_datagenerator.flow_from_dataframe(dataframe=df_train,
                                    directory = train_dir,  
                                    x_col="image_name", 
                                    y_col= df_train.columns.to_list()[2:], 
                                    subset="training", 
                                    batch_size=128,
                                    seed=42, 
                                    shuffle = True, 
                                    class_mode="raw", 
                                    target_size=(128,128))

Found 32384 validated image filenames.


In [11]:
# val_set contains 20 percent of images 
val_datagen = train_datagenerator.flow_from_dataframe(dataframe=df_train,
                            directory = train_dir,  
                            x_col="image_name", 
                            y_col= df_train.columns.to_list()[2:], 
                            subset="validation", 
                            batch_size= 128,
                            seed=42, 
                            shuffle = True, 
                            class_mode="raw", 
                            target_size=(128,128))

Found 8095 validated image filenames.


In [12]:
train_steps = train_datagen.n // train_datagen.batch_size
val_steps = val_datagen.n // train_datagen.batch_size

In [13]:
def fbeta_score(y_true, y_pred, beta = 2, epsilon = 1e-4):
    beta_squared = beta**2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp +fp +epsilon)
    recall = tp/(tp + fn + epsilon)
    
    return (1 + beta_squared) * precision * recall / (beta_squared * precision + recall + epsilon)

def accuracy(y_true, y_pred, epsilon = 1e-4):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
        
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32) * tf.cast(tf.logical_not(y_pred), tf.float32),                 axis = 1)
    return (tp + tn)/(tp + tn + fp + fn + epsilon)

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[fbeta_score,accuracy])

# Save the callback
callback = EarlyStopping(
    monitor="val_loss", 
    patience=2,
    verbose=0,
    baseline=None,
    restore_best_weights=True,
)

In [15]:
history = model.fit(x=train_datagen,
                    steps_per_epoch=train_steps,
                    validation_data=val_datagen,
                    validation_steps=val_steps,
                    epochs=5,
                    callbacks=[callback]) 

Epoch 1/5
253/253 [==============================] - 3430s 14s/step - loss: 0.2446 - fbeta_score: 0.6650 - accuracy: 0.9057 - val_loss: 0.2048 - val_fbeta_score: 0.6970 - val_accuracy: 0.9201
Epoch 2/5
253/253 [==============================] - 2960s 12s/step - loss: 0.1700 - fbeta_score: 0.7755 - accuracy: 0.9336 - val_loss: 0.1564 - val_fbeta_score: 0.7858 - val_accuracy: 0.9376
Epoch 3/5
253/253 [==============================] - 3335s 13s/step - loss: 0.1512 - fbeta_score: 0.8002 - accuracy: 0.9397 - val_loss: 0.1371 - val_fbeta_score: 0.8196 - val_accuracy: 0.9453
Epoch 4/5
253/253 [==============================] - 3429s 14s/step - loss: 0.1404 - fbeta_score: 0.8171 - accuracy: 0.9437 - val_loss: 0.1306 - val_fbeta_score: 0.8287 - val_accuracy: 0.9477
Epoch 5/5
253/253 [==============================] - 3140s 12s/step - loss: 0.1343 - fbeta_score: 0.8302 - accuracy: 0.9467 - val_loss: 0.1269 - val_fbeta_score: 0.8319 - val_accuracy: 0.9500


In [16]:
submission_df = pd.read_csv(os.path.join(path,'sample_submission.csv'))
test_df = submission_df.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)

test_df['image_name'] = submission_df['image_name'].apply(lambda x: x+'.jpg')
test_df.head()

,image_name
0,test_0.jpg
1,test_1.jpg
2,test_2.jpg
3,test_3.jpg
4,test_4.jpg


In [17]:
test_generator = ImageDataGenerator(rescale = 1./255)

test_gen = test_generator.flow_from_dataframe(dataframe=test_df,
                                            directory = os.path.join(path,'test-jpg'),  
                                            x_col="image_name", 
                                            y_col=None, 
                                            batch_size=67,
                                            shuffle=False,
                                            class_mode=None, 
                                            target_size=(128,128))

test_size = test_gen.n // test_gen.batch_size

Found 40669 validated image filenames.


In [18]:
pred = model.predict(test_gen, steps=test_size, verbose=1)

607/607 [==============================] - 3209s 5s/step


In [19]:
labels = df_train['tags'].to_numpy()
unique_labels = list()
for label in labels:
    tags = label.split()
    for tag in tags:
        if tag not in unique_labels:
            unique_labels.append(tag)

In [20]:
test_names = test_gen.filenames 
        
test_result = pd.DataFrame(pred)
test_result = test_result.apply(lambda x: ' '.join(np.array(unique_labels)[x >= 0.2]), axis=1)

test_result_df = pd.DataFrame({'image_name': test_names, 'tags': test_result})
test_result_df.head()

,image_name,tags
0,test_0.jpg,primary clear
1,test_1.jpg,primary clear
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,primary agriculture clear
4,test_4.jpg,primary cloudy partly_cloudy


In [24]:
additional_df = submission_df.iloc[40669:]['image_name'].reset_index().drop('index', axis=1)

In [25]:
additional_df['image_name'] = additional_df['image_name'].apply(lambda x: '{}.jpg'.format(x))
additional_df.head()

,image_name
0,file_0.jpg
1,file_1.jpg
2,file_10.jpg
3,file_100.jpg
4,file_1000.jpg


In [27]:
additional_gen = test_generator.flow_from_dataframe(dataframe=additional_df,
                                                  directory=os.path.join(path,"test-jpg-additional//test-jpg-additional"),
                                                  x_col='image_name',
                                                  y_col = None,
                                                  batch_size=62,
                                                  shuffle=False,
                                                  class_mode=None, 
                                                  target_size=(128, 128))


additional_step = additional_gen.n // additional_gen.batch_size

Found 20522 validated image filenames.


In [28]:
pred_additional = model.predict(additional_gen, steps=additional_step, verbose=1)

331/331 [==============================] - 610s 2s/step


In [29]:
additional_names = additional_gen.filenames 
        
additional_result = pd.DataFrame(pred_additional)
additional_result = additional_result.apply(lambda x: ' '.join(np.array(unique_labels)[x >= 0.2]), axis=1)

additional_result_df = pd.DataFrame({'image_name': additional_names, 'tags': additional_result})
additional_result_df.head()

,image_name,tags
0,file_0.jpg,primary clear
1,file_1.jpg,primary agriculture water road cultivation par...
2,file_10.jpg,haze primary agriculture clear water road
3,file_100.jpg,primary agriculture clear
4,file_1000.jpg,primary clear


In [30]:
final_df = pd.concat([test_result_df, additional_result_df])
final_df = final_df.reset_index().drop('index', axis =1)
final_df

,image_name,tags
0,test_0.jpg,primary clear
1,test_1.jpg,primary clear
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,primary agriculture clear
4,test_4.jpg,primary cloudy partly_cloudy
...,...,...
61186,file_9995.jpg,primary cloudy partly_cloudy
61187,file_9996.jpg,primary clear water
61188,file_9997.jpg,primary clear
61189,file_9998.jpg,haze primary cloudy


In [31]:
final_df['image_name'] = final_df['image_name'].apply(lambda x: x.replace('.jpg',''))
final_df

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary agriculture clear
4,test_4,primary cloudy partly_cloudy
...,...,...
61186,file_9995,primary cloudy partly_cloudy
61187,file_9996,primary clear water
61188,file_9997,primary clear
61189,file_9998,haze primary cloudy


In [32]:
final_df.to_csv('submission.csv', index=False)